<a href="https://colab.research.google.com/github/asrenninger/city-GPT/blob/main/python/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet duckdb pacmap node2vec adjustText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cudnn-

In [2]:
import pacmap

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import duckdb as db

In [100]:
# initialize DuckDB
con = db.connect(database=':memory:')

# load the required extensions
con.execute("INSTALL httpfs;")
con.execute("INSTALL spatial;")
con.execute("LOAD httpfs;")
con.execute("LOAD spatial;")

# fetch data
query = """
          SELECT
            id,
            confidence,
            CAST(names AS JSON)AS names,
            CAST(categories AS JSON) AS categories,
            CAST(brand AS JSON) AS brand,
            CAST(addresses AS JSON) AS addresses,
            CAST(websites AS JSON) AS websites,
            ST_AsText(ST_GeomFromWKB(geometry)) AS geometry
          FROM
            read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/type=*/*', hive_partitioning=1)
          WHERE
            bbox.xmin > -11.02
            AND bbox.xmax < 1.77
            AND bbox.ymin > 49.95
            AND bbox.ymax < 59.74
        """

# execute the query and load data
df = con.execute(query).fetchdf()
df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,id,confidence,names,categories,brand,addresses,websites,geometry
0,08f187293142a411035565e6a3b635d9,0.545360,"{""primary"":""Norrard Rocks"",""common"":null,""rule...","{""main"":""landmark_and_historical_building"",""al...",None,"[{""freeform"":null,""locality"":null,""postcode"":n...",None,POINT (-6.37 49.9533)
1,08f1872931008b0c03541a29324e67b5,0.426304,"{""primary"":""Glenhope & Glenhope High Holiday C...","{""main"":""bed_and_breakfast"",""alternate"":[""cott...",None,"[{""freeform"":""Bryher"",""locality"":""Hugh Town"",""...","[""http://www.visitbryher.co.uk""]",POINT (-6.3571668 49.9510416)
2,08f187293100c4c8032420dd4be9096a,0.667112,"{""primary"":""Bryher, Isles of Scilly"",""common"":...","{""main"":""landmark_and_historical_building"",""al...",None,"[{""freeform"":null,""locality"":null,""postcode"":n...",None,POINT (-6.3575439 49.9513932)
3,08f187293100e813032db62be551544c,0.964027,"{""primary"":""Hell Bay Hotel"",""common"":null,""rul...","{""main"":""hotel"",""alternate"":[""restaurant"",""acc...",None,"[{""freeform"":""Bryher, Isles of Scilly"",""locali...","[""https://linktr.ee/hellbayhotel""]",POINT (-6.3578261 49.9515001)
4,08f187293102cc6803cd90fde71b6134,0.895433,"{""primary"":""The Crab Shack"",""common"":null,""rul...","{""main"":""seafood_restaurant"",""alternate"":[""foo...",None,"[{""freeform"":""Bryher"",""locality"":""Isles Of Sci...","[""https://www.hellbay.co.uk/dining-with-us/cra...",POINT (-6.3562132 49.9524559)


In [101]:
from shapely import wkt
gs = gpd.GeoSeries(df['geometry'].apply(wkt.loads))

In [102]:
gf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")

In [103]:
gf.to_crs("EPSG:4326", inplace=True)

df['lon'] = gf.centroid.x
df['lat'] = gf.centroid.y

gf.to_crs("EPSG:27700", inplace=True)

df['x'] = gf.centroid.x
df['y'] = gf.centroid.y

<ipython-input-103-bc7cfd1129b8>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['lon'] = gf.centroid.x
<ipython-input-103-bc7cfd1129b8>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['lat'] = gf.centroid.y


In [202]:
import json
import ast
import re

In [110]:
json.loads(df['categories'][0])

{'main': 'landmark_and_historical_building',
 'alternate': ['island', 'monument']}

In [112]:
# get the primary name...
def get_name(json_str):
    try:
        data = json.loads(json_str)
        return data['primary'] if data['primary'] else None
    except (KeyError, IndexError, TypeError, json.JSONDecodeError):
        return None

# ...and the first category...
def get_category(json_str):
    try:
        data = json.loads(json_str)
        return data['main'] if data['main'] else None
    except (KeyError, IndexError, TypeError, json.JSONDecodeError):
        return None

# ...and another option.
def get_alternate(json_str):
    try:
        data = json.loads(json_str)
        return data['alternate'][0] if data['alternate'] else None
    except (KeyError, IndexError, TypeError, json.JSONDecodeError):
        return None

# get the brand if there is one.
def get_brand(json_str):
    if not json_str:
        return None
    try:
        # Parse the JSON string
        data = json.loads(json_str)
        # Extract the primary name from the names object
        return data.get('names', {}).get('primary', None)
    except (json.JSONDecodeError, TypeError):
        return None


# get the website of there is one.
def get_website(list_str):
    if not list_str:
        return None
    try:
        data = ast.literal_eval(list_str)
        return data[0] if data else None
    except (ValueError, SyntaxError, IndexError, TypeError):
        return None

# get the country from the address to filter on that.
def get_country(json_str):
    if not json_str:
        return None
    try:
        data = json.loads(json_str)
        return data[0].get('country', None)
    except (json.JSONDecodeError, IndexError, TypeError):
        return None

In [113]:
df['name'] = df['names'].apply(get_name)
df['category'] = df['categories'].apply(get_category)
df['alternate'] = df['categories'].apply(get_alternate)
df['country'] = df['addresses'].apply(get_country)
df['website'] = df['websites'].apply(get_website)
df['brand'] = df['brand'].apply(get_brand)

In [118]:
df = df[
    (df['country']=='GB') & (df['category'].notnull()) & (df['name'].notnull())
    ]
df = df.drop(['addresses', 'websites', 'country'], axis=1)
df = df[['id', 'confidence', 'name', 'category', 'alternate', 'brand', 'website', 'lon', 'lat', 'x', 'y', 'geometry']]
df.head()

,id,confidence,name,category,alternate,brand,website,lon,lat,x,y,geometry
0,08f187293142a411035565e6a3b635d9,0.545360,Norrard Rocks,landmark_and_historical_building,island,None,None,-6.370000,49.953300,86650.738526,15059.727426,POINT (-6.37 49.9533)
1,08f1872931008b0c03541a29324e67b5,0.426304,"Glenhope & Glenhope High Holiday Cottages, Bryher",bed_and_breakfast,cottage,None,http://www.visitbryher.co.uk,-6.357167,49.951042,87556.273753,14754.993784,POINT (-6.3571668 49.9510416)
2,08f187293100c4c8032420dd4be9096a,0.667112,"Bryher, Isles of Scilly",landmark_and_historical_building,island,None,None,-6.357544,49.951393,87531.512714,14795.644919,POINT (-6.3575439 49.9513932)
3,08f187293100e813032db62be551544c,0.964027,Hell Bay Hotel,hotel,restaurant,None,https://linktr.ee/hellbayhotel,-6.357826,49.951500,87511.970445,14808.705331,POINT (-6.3578261 49.9515001)
4,08f187293102cc6803cd90fde71b6134,0.895433,The Crab Shack,seafood_restaurant,food,None,https://www.hellbay.co.uk/dining-with-us/crab-...,-6.356213,49.952456,87633.818654,14908.178983,POINT (-6.3562132 49.9524559)


In [119]:
df[df['website'].notnull()].shape[0] / df.shape[0]

0.8042160954401911

In [159]:
counts = df.groupby('category', as_index=False)[['name']].count().sort_values('name', ascending= False)
bottom = counts[(counts['name'] < 20) & counts['category'].str.contains("_restaurant")]

In [160]:
sub = df.copy()

In [161]:
sub['category'] = sub['category'].apply(lambda x: 'other_restaurant' if x in bottom.category.tolist() else x)

In [162]:
cut = counts[counts['name'] < 20]

In [163]:
def recode(cat_str, val_cat):
  """
  make all strings that contain _museum into just 'museum'...
  ...and all strings that contain _club into just 'club.

  else return the original string
  """
  if cat_str is None:
    return 'other'

  if cat_str not in val_cat:
    return cat_str

  if 'museum' in cat_str:
    return 'museum'
  elif '_grocery' in cat_str:
    return 'specialty_grocery_store'
  elif '_bar' in cat_str:
    return 'bar'
  elif '_clinic' in cat_str:
    return 'clinic'
  elif 'b2b' in cat_str:
    return 'b2b'
  elif 'other_stadium' in cat_str:
    return 'stadium'
  else:
    return cat_str

sub['category'] = sub['category'].apply(lambda x: recode(x, cut.category.tolist()))

In [170]:
counts = sub.groupby('category', as_index=False)[['name']].count().sort_values('name', ascending= False)
bottom = counts[(counts['name'] < 10)]

In [172]:
sub = sub[~sub['category'].isin(bottom.category.tolist())]

In [223]:
def clean_name(name):
    if not isinstance(name, str):
        return name

    # Step 1: Remove parentheticals
    name = re.sub(r'\(.*?\)', '', name)
    # Step 2: Replace all caps ATM with ATM
    name = re.sub(r'\bATM\b', 'ATM', name)
    # Step 3: Replace "Iceland" with "Iceland Foods"
    if name.strip() == "Iceland":
        name = "Iceland Foods"
    # Step 4: Replace "Boots" with "Boots Pharmacy"
    if name.strip() == "Boots":
        name = "Boots Pharmacy"
    # Step 5: Replace anything with "Boots Opticians" with "Boots Opticians"
    if "Boots Opticians" in name:
        name = "Boots Opticians"
    # Step 6: Replace instances of m and s or m&s with M&S (not case sensitive)
    name = re.sub(r'\bm[&\s]?s\b', 'M&S', name, flags=re.IGNORECASE)
    # Step 7: Remove stray GB or UK or UK & Ireland with leading spaces
    name = re.sub(r'\s+(GB|UK|UK & Ireland)', '', name)
    # Step 8: Replace anything with "Royal Mail" with "Royal Mail"
    if "Royal Mail" in name:
        name = "Royal Mail"
    # Step 9: Replace anything with "Domino's Pizza" with "Domino's Pizza"
    if "Domino's Pizza" in name:
        name = "Domino's Pizza"
    # Step 10: Replace anything with "Pizza Hut" with "Pizza Hut"
    if "Pizza Hut" in name:
        name = "Pizza Hut"
    # Step 11: Replace "Company" at the end with "Co"
    name = re.sub(r'\bCompany\b$', 'Co', name)
    # Step 12: Replace anything with "Pret a Manger" with "Pret a Manger" (case insensitive)
    if re.search(r'Pret a Manger', name, re.IGNORECASE):
        name = re.sub(r'Pret a Manger.*', 'Pret a Manger', name, flags=re.IGNORECASE)
    # Step 13: Strip anything after "M&S" except "Cafe"
    if name.startswith("M&S"):
        if "Cafe" in name:
            name = "M&S Cafe"
        else:
            name = "M&S"
    # Step 14: Replace any entry containing "Argos" with "Argos"
    if "Argos" in name:
        name = "Argos"
    # Step 15: Strip out any special characters at the end of the name
    name = re.sub(r'[^\w\s]$', '', name)

    # Remove extra spaces
    name = name.strip()
    return name

In [224]:
sub['name'] = sub['name'].apply(clean_name)

In [227]:
pub_names = [
    "Red Lion",
    "The Crown",
    "Royal Oak",
    "White Hart",
    "The Swan",
    "The Plough",
    "The Bell",
    "Rose & Crown",
    "Queens Head",
    "Railway Tavern",
    "The Ship",
    "Kings Arms",
    "White Horse",
    "Kings Head",
    "Chequers",
    "Rising Sun",
    "The George",
    "Fox & Hounds",
    "Prince Of Wales",
    "Black Horse",
    "The Fox",
    "Cross Keys",
    "The Star",
    "Three Horseshoes",
    "The Greyhound",
    "Coach & Horses",
    "The Victoria",
    "George & Dragon",
    "Masons Arms",
    "Hare & Hounds",
    "White Lion",
    "The Sun",
    "White Swan",
    "Nags Head",
    "Carpenters Arms",
    "Duke Of York",
    "Cricketers",
    "Windmill",
    "Black Lion",
    "Travellers Rest",
    "Station Hotel",
    "Golden Lion",
    "The George",
    "Bird In Hand",
    "Black Bull",
    "Horse & Groom",
    "Butchers Arms",
    "Bulls Head",
    "The Beehive",
    "The Anchor",
    "The Castle",
    "Red Dragon",
    "The Eagle",
    "The Green Man",
    "The Royal George",
    "The King's Head",
    "The Fox & Hound",
    "The Rose",
    "The Jolly Farmer",
    "The Old Mill",
    "The Ship Inn",
    "The Admiral",
    "The Old Bell",
    "The Queen's Arms",
    "The Crown & Anchor",
    "The White Lion",
    "The Black Swan",
    "The Cherry Tree",
    "The Blue Boar",
    "The Sun Inn",
    "The Bear",
    "The White Bear",
    "The Three Tuns",
    "The Greyhound Inn",
    "The Green Dragon",
    "The Bull",
    "The Lamb",
    "The Dog & Duck",
    "The Kings Head",
    "The Saracens Head",
    "The New Inn",
    "The Farmers Arms",
    "The Seven Stars",
    "The Bluebell",
    "The Red Hart",
    "The Old Crown",
    "The Wellington",
    "The Griffin",
    "The Unicorn",
    "The Fox Inn",
    "The Rising Sun Inn",
    "The Kings Arms Hotel",
    "The Old Ship",
    "The Duke Of Wellington",
    "The Black Boy",
    "The George & Dragon Inn",
    "The Bull & Bush",
    "The Golden Fleece",
    "The Peasant",
    "The Cricketers Arms"
]

In [243]:
sub['name'] = sub.apply(lambda row: "Classic Pub" if row['name'] in pub_names and row['category'] in ['pub', 'bar'] else row['name'], axis=1)
mask = sub['name'].map(sub['name'].value_counts()) > 9
sub['label'] = np.where(mask, sub['name'], sub['category'])

In [244]:
sub['label'] = sub['label'].str.replace("_", " ")

In [245]:
sub = sub.dropna(subset=['label'])

In [246]:
sub.shape

(2054931, 13)

In [254]:
sub

,id,confidence,name,category,alternate,brand,website,lon,lat,x,y,geometry,label
0,08f187293142a411035565e6a3b635d9,0.545360,Norrard Rocks,landmark_and_historical_building,island,None,None,-6.370000,49.953300,86650.738526,15059.727426,POINT (-6.37 49.9533),landmark and historical building
1,08f1872931008b0c03541a29324e67b5,0.426304,"Glenhope & Glenhope High Holiday Cottages, Bryher",bed_and_breakfast,cottage,None,http://www.visitbryher.co.uk,-6.357167,49.951042,87556.273753,14754.993784,POINT (-6.3571668 49.9510416),bed and breakfast
2,08f187293100c4c8032420dd4be9096a,0.667112,"Bryher, Isles of Scilly",landmark_and_historical_building,island,None,None,-6.357544,49.951393,87531.512714,14795.644919,POINT (-6.3575439 49.9513932),landmark and historical building
3,08f187293100e813032db62be551544c,0.964027,Hell Bay Hotel,hotel,restaurant,None,https://linktr.ee/hellbayhotel,-6.357826,49.951500,87511.970445,14808.705331,POINT (-6.3578261 49.9515001),hotel
4,08f187293102cc6803cd90fde71b6134,0.895433,The Crab Shack,seafood_restaurant,food,None,https://www.hellbay.co.uk/dining-with-us/crab-...,-6.356213,49.952456,87633.818654,14908.178983,POINT (-6.3562132 49.9524559),seafood restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432709,08f19401989955210346799e26e3ad58,0.959385,PMR Vehicle Repairs Withernsea,automotive_repair,automotive,None,http://www.pmr.autos,0.010345,53.745169,532674.002520,429518.998832,POINT (0.0103448 53.7451686),automotive repair
2432710,08f194019899552103d95e5b320986c7,0.363008,Lola Rose Florals & Gifts,arts_and_crafts,None,None,None,0.010345,53.745169,532674.002520,429518.998832,POINT (0.0103448 53.7451686),arts and crafts
2432711,08f194019899552103a96fae0b0e1d44,0.722789,C &I Walker Furniture sales/clearance/removals,shopping,furniture_store,None,None,0.010345,53.745169,532674.002520,429518.998832,POINT (0.0103448 53.7451686),shopping
2432713,08f19401a565b7880321f1fbc897426b,0.667112,Westermost Rough Wind Farm,landmark_and_historical_building,campus_building,None,None,0.149000,53.805000,541615.226641,436442.093870,POINT (0.149 53.805),landmark and historical building


In [256]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [257]:
sub.drop(['geometry'], axis=1).to_csv("drive/MyDrive/overture/overture_cleaned.csv")